In [52]:
import numpy as np
import pandas as pd
import string
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import StandardScaler ,Normalizer

import tensorflow as tf

In [53]:
data=pd.read_csv("/content/spotify_songs.csv")
data

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,...,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [55]:
# these columns don't have any useful information,so we can drop them.
data.drop(["track_id","track_album_id","playlist_id","track_album_release_date"],axis=1,inplace=True)


In [56]:
# drop null values.
data.dropna(inplace=True)
data.isnull().sum()

track_name           0
track_artist         0
track_popularity     0
track_album_name     0
playlist_name        0
playlist_genre       0
playlist_subgenre    0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
dtype: int64

In [57]:
# drop duplicated rows
data.drop_duplicates(keep="first",inplace=True)
data.drop_duplicates(subset=["track_name"],inplace=True)
data.shape

(23449, 19)

Categorical dataset

In [58]:
categorical_data=data.select_dtypes("object")
categorical_data

,track_name,track_artist,track_album_name,playlist_name,playlist_genre,playlist_subgenre
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,pop,dance pop
1,Memories - Dillon Francis Remix,Maroon 5,Memories (Dillon Francis Remix),Pop Remix,pop,dance pop
2,All the Time - Don Diablo Remix,Zara Larsson,All the Time (Don Diablo Remix),Pop Remix,pop,dance pop
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,Call You Mine - The Remixes,Pop Remix,pop,dance pop
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,Someone You Loved (Future Humans Remix),Pop Remix,pop,dance pop
...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,edm,progressive electro house
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,Closer Remixed,♥ EDM LOVE 2020,edm,progressive electro house
32830,Sweet Surrender - Radio Edit,Starkillers,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,edm,progressive electro house
32831,Only For You - Maor Levi Remix,Mat Zo,Only For You (Remixes),♥ EDM LOVE 2020,edm,progressive electro house


In [59]:
# merging columns

categorical_data["song_description"]=categorical_data["track_name"]+categorical_data["track_album_name"]+categorical_data["playlist_name"]+categorical_data["playlist_subgenre"]

categorical_data["description"]=categorical_data["track_artist"]+" "+categorical_data["song_description"]

In [60]:
categorical_data.drop(["track_name","track_album_name","playlist_subgenre","playlist_name","song_description","track_artist"],axis=1,inplace=True)

In [61]:
categorical_data.rename(columns={"playlist_genre":"labels"},inplace=True)
categorical_data

,labels,description
0,pop,Ed Sheeran I Don't Care (with Justin Bieber) -...
1,pop,Maroon 5 Memories - Dillon Francis RemixMemori...
2,pop,Zara Larsson All the Time - Don Diablo RemixAl...
3,pop,The Chainsmokers Call You Mine - Keanu Silva R...
4,pop,Lewis Capaldi Someone You Loved - Future Human...
...,...,...
32828,edm,Lush & Simon City Of Lights - Official Radio E...
32829,edm,Tegan and Sara Closer - Sultan & Ned Shepard R...
32830,edm,Starkillers Sweet Surrender - Radio EditSweet ...
32831,edm,Mat Zo Only For You - Maor Levi RemixOnly For ...


In [62]:
def clean(text):
    #HTML to text format
    soup=BeautifulSoup(text)
    text=soup.getText()
    return text

In [63]:
categorical_data["description"]=categorical_data["description"].apply(clean)

<ipython-input-62-4cd92a818bdd>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup=BeautifulSoup(text)


In [64]:
label_encoder=LabelEncoder()
categorical_data["labels"]=label_encoder.fit_transform(categorical_data["labels"])

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [66]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(categorical_data['description'])

In [67]:
# number of unique words
vocab_len=len(tokenizer.word_index)+1
vocab_len

47778

In [68]:
encoded_texts=tokenizer.texts_to_sequences(categorical_data['description'])

In [69]:
# picking up the longest encoded sequence
length_list=[]
for seq in encoded_texts:
  length_list.append(len(seq))

max_len=max(length_list)

In [70]:
max_len

58

In [71]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [72]:
X=pad_sequences(encoded_texts,maxlen=max_len,padding="pre")

In [73]:
# 23449 samples and 58 features

X.shape

(23449, 58)

In [74]:
# extractind labels
y=categorical_data["labels"]


In [75]:
categorical_data=pd.DataFrame(X)
categorical_data_mat=categorical_data.to_numpy()

Numerical dataset

In [76]:
numerical_data = data.select_dtypes(["float64","int64"])

In [77]:
numerical_data["duration_ms"]=numerical_data["duration_ms"]/60000

In [78]:

scaler=StandardScaler()
numerical_data=scaler.fit_transform(numerical_data)
numerical_data=pd.DataFrame(numerical_data)
norm=Normalizer(norm="l2")
numerical_data=norm.fit_transform(numerical_data)

In [79]:
numerical_data=pd.DataFrame(numerical_data)
numerical_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000,23449.000000
mean,0.009141,0.013048,0.017732,-0.000146,0.024645,0.003371,-0.019350,-0.022565,-0.030964,-0.017370,0.007478,-0.007053,-0.012472
std,0.298749,0.281817,0.270367,0.304008,0.258867,0.316236,0.266439,0.262023,0.231633,0.258471,0.293647,0.284292,0.260929
min,-0.736551,-0.796255,-0.777393,-0.719985,-0.914120,-0.714491,-0.471250,-0.517032,-0.279917,-0.515800,-0.795081,-0.762611,-0.753423
25%,-0.216635,-0.188248,-0.196455,-0.282069,-0.142022,-0.330749,-0.200285,-0.213234,-0.137669,-0.194291,-0.228201,-0.241680,-0.196809
50%,0.037238,0.035365,0.040803,0.039426,0.057705,0.188754,-0.122106,-0.130098,-0.115346,-0.111253,0.002369,0.011779,-0.050020
75%,0.239850,0.223998,0.243138,0.252183,0.220126,0.280777,0.083105,0.115439,-0.083520,0.113876,0.238762,0.146407,0.138333
max,0.738870,0.745852,0.696681,0.747516,0.679523,0.615844,0.937662,0.829024,0.894179,0.923020,0.739478,0.843832,0.910237


In [80]:
numerical_data_mat=numerical_data.to_numpy()
numerical_data_mat.shape

(23449, 13)

In [81]:
final_data=np.concatenate([numerical_data_mat,categorical_data_mat],axis=1)

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X=final_data


In [84]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)